In [ ]:
import pandas as pd
import numpy as np
from io import StringIO

Files imported from SIS: Students_export.csv, Studentrace.csv <br>
Files imported from StudentContact_Total3 script: StudentContactsBeta.csv
File exported: eIEPPROStudentDemographics.csv

<H4> Things to do: <br>
* encapsulate all text in quotes (needs to be done) <br>
* Add student demographic info & Parent/Guardian contacts afterwards

<H5> StudentRace Table: converted to binary values 
    
| STUDENTRACE.StudentID |STUDENTRACE.RaceCd | ---|
|--- |--- |--- |
| Some# | Possibilities are: |
| AS | Asian | 
| WH | White | 
| BL | Black | 
| AM | American Native | 
| PI | Pacific Islander | 
| not in Student Race table | Hispanic |

Appendix 1 Spec (06/2022 Document v22)

| eIEPPRO Data Field | Powerschool Table Name | Details | 
| --- |--- |--- |
| StateIDNumber | STUDENTS.State_StudentNumber | nvarchar (10) |
| SISIDNumber   | STUDENTS.Student_Number      | nvarchar(9)   |
| SchoolCode    | STUDENTS.SchoolID            | nvarchar(9)   |  
| LastName      | STUDENTS.Last_Name           | nvarchar(40)  |
| FirstName     | STUDENTS.First_Name          | nvarchar(25)  |
| MiddleName    | STUDENTS.Middle_Name         | nvarchar(25)  |
| DOB           | STUDENTS.DOB                 | smalldatetime |
| Gender        | STUDENTS.Gender              | nvarchar(6)*  |  
| Ethnicity_HI  | STUDENTS.Hispanic Y/N        | Yes: HI == 1 |
| Ethnicity_AM  | STUDENTRACE.RaceCd == AM |--- |
| Ethnicity_AS  | STUDENTRACE.RaceCd == AS |--- |
| Ethnicity_BL  | STUDENTRACE.RaceCd == BL |--- |
| Ethnicity_PI  | STUDENTRACE.RaceCd == PI |--- |
| Ethnicity_WH  | STUDENTRACE.RaceCd == AM |--- |
| Grade | STUDENTS.Grade_Level |--- |
| Student Address #1          | STUDENTS.Mailing_Street     | Powerschool  ~ Street Address |
| Student Address #2          | Students.Mailing_City, Mailing_State and Mailing_Zip     |  |
| Student Physical Address ~ StudentPhyAddress  | STUDENTS.Street   | STUDENTS.Street |  
| Student Phone #             | StudentPhone        | STUDENTS.Home_Phone |
| Student E-Mail              | StudentEmail        | will not provide |
| Parent #1 Name              | Parent1Name         | --- |
| Parent #1 Address-1         | Parent1Address1     | Street Address |
| Parent #1 Address-2         | Parent1Address2     | City, State and Zip Code |  
| Parent #1 Home Phone        | Parent1HomePhone    | --- |
| Parent #1 Work Phone        | Parent1WorkPhone    | --- |
| Parent #1 Cell Phone        | Parent1CellPhone    | --- |
| Parent #1 Relationship      | Parent1Relationship | --- |
| Parent #1 Email             | Parent1Email        | --- |
| Parent #2 Name              | Parent2Name         | --- |
| Parent #2 Address-1         | Parent2Address1     | Street Address |
| Parent #2 Address-2         | Parent2Address2     | City, State and Zip Code |  
| Parent #2 Home Phone        | Parent2HomePhone    | --- |
| Parent #2 Work Phone        | Parent2WorkPhone    | --- |
| Parent #2 Cell Phone        | Parent2CellPhone    | --- |
| Parent #2 Relationship      | Parent2Relationship | --- |
| Parent #2 Email             | Parent2Email        | --- |
| PrimaryLanguage | S_AZ_STU_X.spokenlanguage |--- |
| HomeLanguage | S_AZ_STU_X.homelanguagecode |--- |
| District of Residence | DOR |--- |
| District of Attendance | DOA |--- |
| Home School | HomeSchool |--- |
| Attending School | AttendingSchool |--- |
| Emergency Contact | EmergencyContact |--- |
| Emergency Contact Phone | EmergencyContactPhone |--- |

* Gender: AzEDS defined Gender values <br>
* Students pulled based on ACTIVITIES.childfind504 == 1 or ACTIVITIES.childfindidea == 1 <br>
S_AZ_STU_X.firstacquiredlanguage <br>
S_AZ_STU_X.homelanguagecode is home language <br>
S_AZ_STU_X.spokenlanguage is the same as primary language <br>


<H3> SIS Input Files: <br>

In [ ]:
Students = pd.read_csv("Students_export.csv", encoding='latin-1')
StudentRaceAndEthnicity = pd.read_csv("StudentRace.csv", encoding='latin-1')

<H3> Script Input Files: <br>

In [ ]:
# Leaving out for now
# StudentContacts = pd.read_csv("StudentContactsBeta.csv", encoding='latin-1')

In [ ]:
# The e-IEPPRO spec has fields for parent addresses that our SIS doesn't export
# StudentContacts

<H5> All students with the flags for childfind504 and childfindidea are pulled in. These flags are to be kept on for the time that students have a 504 or IEP. <br>
    
<H4> Consider running tests to compare Flag population to Population at Large - compare numbers

In [ ]:
Students_504 = Students.loc[Students['ACTIVITIES.childfind504'] == 1] 
Students_and_idea = Students.loc[Students['ACTIVITIES.childfindidea'] == 1]

In [ ]:
# 630 students on 01/03/23
Students_504

In [ ]:
# 2056 students on 01/03
Students_and_idea

In [ ]:
# Combine communities and drop duplicates
df_504_id = pd.concat([Students_504, Students_and_idea], ignore_index=True).drop_duplicates()

In [ ]:
# 2672 combined on 01/03
df_504_id

In [ ]:
# Clean columns
df_504_id.columns = df_504_id.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('/', '')

In [ ]:
# Create the dataframe for writing, column by column
df_StudentData = df_504_id.filter(['students.state_studentnumber', 
                                  'students.student_number',
                                  'students.id',
                                  'students.schoolid',
                                  'students.last_name',
                                  'students.first_name',
                                  'students.middle_name',
                                  'students.dob',
                                  'students.gender',
                                  'hispanic_yn', #insert red data later                   
                                  'students.grade_level',
                                  'students.mailing_street',
                                  'students.mailing_city',
                                  'students.mailing_state',
                                  'students.mailing_zip',
                                  'students.street', # Using for physical street
                                  'students.home_phone', #Considering inserting contrived email
                                  's_az_stu_x.homelanguagecode',
                                  's_az_stu_x.spokenlanguage',
                                  'ell_alert',
                                  '504_alert'
                                 ])

In [ ]:
df_StudentData.set_index('students.student_number')

In [ ]:
df_StudentData

In [ ]:
df_StudentData.info()

<H5> Transforming Columns

In [ ]:
# Clean data as appropriate
# Turn State_StudentNumber to varchar
# df["Fee"]=df["Fee"].values.astype('string')
df_StudentData["stu_id"]        = df_StudentData["students.id"].values.astype('int')
df_StudentData["StateIDNumber"] = df_StudentData["students.state_studentnumber"].values.astype('int')
df_StudentData["SISIDNumber"]   = df_StudentData["students.state_studentnumber"].values.astype('int') # as requested
df_StudentData["SchoolCode"]    = df_StudentData["students.schoolid"].values.astype('int')
df_StudentData["LastName"]      = df_StudentData["students.last_name"]
df_StudentData["FirstName"]     = df_StudentData["students.first_name"]
df_StudentData["MiddleName"]    = df_StudentData["students.middle_name"]
df_StudentData["DOB"]           = pd.to_datetime(df_StudentData["students.dob"])
df_StudentData["Gender"]        = df_StudentData["students.gender"]
df_StudentData["Ethnicity_HI"]  = df_StudentData["hispanic_yn"]
# Break to insert RED Data

df_StudentData["Grade"]           = df_StudentData["students.grade_level"].values.astype('int')
df_StudentData["StudentAddress1"] = df_StudentData["students.mailing_street"]
df_StudentData["StudentAddress2"] = df_StudentData['students.mailing_city'].astype(str) + ', ' + df_StudentData['students.mailing_state'] + ', ' + df_StudentData['students.mailing_zip']
df_StudentData["StudentPhyAddress"] = df_StudentData["students.street"]
df_StudentData["StudentPhone"]      = df_StudentData["students.home_phone"]
df_StudentData["StudentEMail"]      = ""

# Insert Parent Contact Data
df_StudentData["Parent1Name"]              = ""
df_StudentData["Parent1Address1"]          = ""
df_StudentData["Parent1Address2"]          = ""
df_StudentData["Parent1HomePhone"]         = ""
df_StudentData["Parent1WorkPhone"]         = ""
df_StudentData["Parent1CellPhone"]         = ""
df_StudentData["Parent1Relationship"]      = ""
df_StudentData["Parent1Email"]             = ""
df_StudentData["Parent2Name"]              = ""
df_StudentData["Parent2Address1"]          = ""
df_StudentData["Parent2Address2"]          = ""
df_StudentData["Parent2HomePhone"]         = ""
df_StudentData["Parent2WorkPhone"]         = ""
df_StudentData["Parent2CellPhone"]         = ""
df_StudentData["Parent2Relationship"]      = ""
df_StudentData["Parent2Email"]             = ""

df_StudentData['VisionTestDate'] = ""
df_StudentData['HearingTestDate'] = ""
df_StudentData['PrimaryLangTestDate'] = ""
df_StudentData['VisionTestResults'] = ""
df_StudentData['HearingTestResults'] = ""
df_StudentData['PrimaryLangTestResults'] = ""
df_StudentData['AZELLATestDate'] = ""
df_StudentData['AZELLATestResults'] = ""

df_StudentData["PrimaryLanguage"]  = df_StudentData["s_az_stu_x.spokenlanguage"] #.values.astype('str') # clearing out int to see what happens
df_StudentData["HomeLanguage"]     = df_StudentData["s_az_stu_x.homelanguagecode"] # .values.astype('int')



<H5> Primary and Home Language Transformations:

In [ ]:
transform_language = {
    0:   'English', 
    1:   'Spanish', 
    2:   'Arabic', 
    3:   'Cambodian', 
    4:   'Cantonese', 
    5:   'Czechoslovakian', 
    6:   'Dutch', 
    7:   'Filipino', 
    8:   'Finnish', 
    9:   'French', 
    10:   'German', 
    11:   'Greek', 
    12:   'Hebrew', 
    13:   'Hungarian', 
    14:   'Italian', 
    15:   'Japanese', 
    16:   'Korean', 
    17:   'Laotian', 
    18:   'Mandarin', 
    19:   'Polish', 
    20:   'Portuguese', 
    21:   'Romanian', 
    22:   'Russian', 
    23:   'Thai', 
    24:   'Ukrainian', 
    25:   'Vietnamese', 
    26:   'Serbo-Croatian', 
    27:   'Other Non-Indian', 
    28:   'Navajo', 
    29:   'Apache (San Carlos)', 
    30:   'Apache (Whiteriver)', 
    31:   'Chemehuevi', 
    32:   'Cocopah', 
    33:   'Hopi', 
    34:   'Haulapai', 
    35:   'Kaibab-Paiute', 
    36:   'Maricopa', 
    37:   'Mohave', 
    38:   'Papago (Tohono Oodham)',
    39:   'Pima (Akimel Oodhman)',
    40:   'Supai', 
    41:   'Tewa', 
    42:   'Yaqui', 
    43:   'Yavapai', 
    44:   'Yuma', 
    45:   'Other Indian', 
    46:   'American Sign Language',
    47:   'Albanian', 
    48:   'Amharic', 
    49:   'Azeri', 
    50:   'Burmese', 
    51:   'Dari', 
    52:   'Dinka', 
    53:   'Farsi', 
    54:   'Kirundi', 
    55:   'Kru', 
    56:   'Lingala', 
    57:   'Maay', 
    58:   'Mandingo', 
    59:   'Nuer', 
    60:   'Pashto', 
    61:   'Serbian', 
    62:   'Somali', 
    63:   'Sawhili', 
    64:   'Tagalog', 
    65:   'Urdu', 
    66:   'Uzbek', 
    67:   'Vai', 
    68:   'Afrikaans', 
    69:   'Bengali', 
    70:   'Hindi', 
    71:   'Persian', 
    72:   'Punjabi', 
    73:   'Cherokee', 
    74:   'Armenian', 
    75:   'Bosnian', 
    76:   'Central Khmer', 
    77:   'Chuukese', 
    78:   'Gujarati', 
    79:   'Haitian; Haitian Creole', 
    80:   'Hmong; Mong', 
    81:   'Karen Languages', 
    82:   'Lao', 
    83:   'Malayalam', 
    84:   'Marshallese', 
    85:   'Nepali', 
    86:   'Otomian languages', 
    87:   'Samoan', 
    88:   'Tamil', 
    89:   'Telugu', 
    90:   'Tigrinya', 
    91:   'Turkish', 
    92:   'undeclared', 
    93:   'Twi', 
    94:   'Yupik languages'
}

df_StudentData["PrimaryLanguage"] = df_StudentData["PrimaryLanguage"].map(transform_language)
df_StudentData["HomeLanguage"] = df_StudentData["HomeLanguage"].map(transform_language)

<H5> District Transformations: <br>
District of Residence & Attendance: based on School ID, but LOA gets Surprise (double check)

In [ ]:
df_StudentData['SchoolCode'].value_counts()

In [ ]:
transform_schoolID = {
    100: 'Legacy Traditional School - Maricopa',
    150: 'Legacy Traditional School - Casa Grande',
    200: 'Legacy Traditional School - Queen Creek',
    250: 'Legacy Traditional School - Avondale',
    300: 'Legacy Traditional School - Chandler',
    350: 'Legacy Traditional School - NW Tucson',
    400: 'Legacy Traditional School - Laveen',
    450: 'Legacy Traditional School - Gilbert',
    500: 'Legacy Traditional School - Surprise',
    550: 'Legacy Traditional School - North Chandler',
    600: 'Legacy Traditional School - Glendale',
    650: 'Legacy Traditional School - Peoria',
    700: 'Legacy Traditional School - East Mesa',
    750: 'Legacy Traditional School - Phoenix',
    800: 'Legacy Traditional School - Goodyear',
    850: 'Legacy Traditional School - North Phoenix',
    900: 'Legacy Traditional School - West Surprise',
    950: 'Legacy Traditional School - East Tucson',
    1000: 'Legacy Traditional School - Deer Valley',
    1050: 'Legacy Traditional School - Mesa',
    1100: 'Legacy Online Academy', 
    1150: 'Legacy Traditional School - San Tan'
}

# Home School and Attending School his is where LOA stays LOA - plus students that are exceptions to the rule?
df_StudentData['HomeSchool'] = df_StudentData['SchoolCode'].map(transform_schoolID)
df_StudentData['AttendingSchool'] = df_StudentData['SchoolCode'].map(transform_schoolID)


In [ ]:
# This where LOA should get Surprise data
# District of Residence and District of Attendance, for all LOA students, is Surprise
# Home School and Attending School keep LOA intact
df_StudentData['DOR_1'] = df_StudentData['SchoolCode']
df_StudentData['DOA_1'] = df_StudentData['SchoolCode']

# data["Age"]= data["Age"].replace(25.0, "Twenty five")
df_StudentData['DOR_2'] = df_StudentData['DOR_1'].replace(1100, 500)
df_StudentData['DOA_2'] = df_StudentData['DOR_1'].replace(1100, 500)

transform_schoolID = {
    100: 'Legacy Traditional School - Maricopa',
    150: 'Legacy Traditional School - Casa Grande',
    200: 'Legacy Traditional School - Queen Creek',
    250: 'Legacy Traditional School - Avondale',
    300: 'Legacy Traditional School - Chandler',
    350: 'Legacy Traditional School - NW Tucson',
    400: 'Legacy Traditional School - Laveen',
    450: 'Legacy Traditional School - Gilbert',
    500: 'Legacy Traditional School - Surprise',
    550: 'Legacy Traditional School - North Chandler',
    600: 'Legacy Traditional School - Glendale',
    650: 'Legacy Traditional School - Peoria',
    700: 'Legacy Traditional School - East Mesa',
    750: 'Legacy Traditional School - Phoenix',
    800: 'Legacy Traditional School - Goodyear',
    850: 'Legacy Traditional School - North Phoenix',
    900: 'Legacy Traditional School - West Surprise',
    950: 'Legacy Traditional School - East Tucson',
    1000: 'Legacy Traditional School - Deer Valley',
    1050: 'Legacy Traditional School - Mesa',
    1100: 'Legacy Online Academy', 
    1150: 'Legacy Traditional School - San Tan'
}

# Home School and Attending School his is where LOA stays LOA - plus students that are exceptions to the rule?

df_StudentData['DOR'] = df_StudentData['DOR_2'].map(transform_schoolID)
df_StudentData['DOA'] = df_StudentData['DOA_2'].map(transform_schoolID)

In [ ]:
df_StudentData['DOA'].value_counts()

In [ ]:
df_StudentData['DOR'].value_counts()

In [ ]:
df_StudentData['EmergencyContact'] = ''
df_StudentData['EmergencyContactPhone'] = ''

<H4> Last Fields from Schools_Export - these students should have ChildFing Flags preserved: <br>
    * 504Student (comes from '504 Alert'), comes in as Y, translate to 1 <br> 
    * EL Student (comes from 'ELL Alert') <br>

In [ ]:
# sampleDF.housing.replace(('yes', 'no'), (1, 0), inplace=True)
df_StudentData["504Student"]       = df_StudentData['504_alert']
df_StudentData['504Student'].replace(('Y', ''), (1, 0), inplace=True)

In [ ]:
df_StudentData["ELStudent"]        = df_StudentData['ell_alert']

<H3> Student Race and Ethnicity data <br>
<H5> HI, AM, AS, BL, PI, WH must be converted to 1 = YES or 0 = No values, per student <br>
    based on STUDENTRACE.StudentID <br>
    STUDENTS.ID == STUDENTRACE.StudentID <br>

In [ ]:
# Create dataframe - this version works!
def BuildRaceMatrix8( df_StudentRace ):
    global df_RED
    df_RED = pd.DataFrame()
    """ Create a counter for each student ID """
    for index, row in df_StudentRace.iterrows():
        student_ID = row['STUDENTRACE.StudentID']
        race_data = row['STUDENTRACE.RaceCd']
        df_RED.at[index, 'stu_id'] = student_ID
        
        match race_data:
            case 'AM':
                df_RED.at[index, 'Ethnicity_AM'] = 1
            case 'AS':
                df_RED.at[index, 'Ethnicity_AS'] = 1
            case 'BL':
                df_RED.at[index, 'Ethnicity_BL'] = 1
            case 'PI':
                df_RED.at[index, 'Ethnicity_PI'] = 1
            case 'WH':
                df_RED.at[index, 'Ethnicity_WH'] = 1
    return df_RED

In [ ]:
testRed3 = BuildRaceMatrix8(StudentRaceAndEthnicity).fillna(0).astype(int)

In [ ]:
df_pivot = pd.pivot_table(testRed3, values=['stu_id', 'Ethnicity_AS', 'Ethnicity_WH', 'Ethnicity_BL', 'Ethnicity_AM', 'Ethnicity_PI'], index=['stu_id'], aggfunc=np.sum, margins=True)

In [ ]:
df_Pivot_plus_StudentData = df_StudentData.join(df_pivot, on='stu_id')

<H5> Bring in Student Contacts

In [ ]:
df_Pivot_plus_StudentData.info()

<H4> The dataframe that will be uploaded - with blank fields

In [ ]:
df_write = df_Pivot_plus_StudentData[['StateIDNumber', 'SISIDNumber', 'SchoolCode', 'LastName', 'FirstName', 'MiddleName',
                                      'DOB','Gender', 'Ethnicity_HI', 'Ethnicity_AM', 'Ethnicity_AS', 'Ethnicity_BL', 'Ethnicity_PI', 'Ethnicity_WH', 'Grade', 'StudentAddress1',
                                      'StudentAddress2', 'StudentPhyAddress', 'StudentPhone', 'StudentEMail', 'Parent1Name', 'Parent1Address1', 'Parent1Address2',
                                      'Parent1HomePhone', 'Parent1WorkPhone', 'Parent1CellPhone', 'Parent1Relationship', 'Parent1Email',
                                      'Parent2Name', 'Parent2Address1', 'Parent2Address2',
                                      'Parent2HomePhone', 'Parent2WorkPhone', 'Parent2CellPhone', 'Parent2Relationship', 'Parent2Email',
                                      'PrimaryLanguage', 'HomeLanguage', 'DOR', 'DOA', 'HomeSchool', 'AttendingSchool', 'EmergencyContact', 
                                      'EmergencyContactPhone', 'VisionTestDate', 'HearingTestDate', 'PrimaryLangTestDate', 'VisionTestResults', 'HearingTestResults',
                                      'PrimaryLangTestResults', 'AZELLATestDate', 'AZELLATestResults',
                                      '504Student', 'ELStudent'
                                     ]]

<H4> The dataframe that will be validated - no blanks

In [ ]:
df_write_noblanks = df_Pivot_plus_StudentData[['StateIDNumber', 'SISIDNumber', 'SchoolCode', 'LastName', 'FirstName', 'MiddleName',
                                      'DOB','Gender', 'Ethnicity_HI', 'Ethnicity_AM', 'Ethnicity_AS', 'Ethnicity_BL', 'Ethnicity_PI', 'Ethnicity_WH', 'Grade', 'StudentAddress1',
                                      'StudentAddress2', 'StudentPhyAddress', 'StudentPhone', 
                                      'PrimaryLanguage', 'HomeLanguage', 'DOR', 'DOA', 'HomeSchool', 'AttendingSchool', 
                                      '504Student', 'ELStudent'
                                     ]]

In [ ]:
df_write_noblanks.info()

In [ ]:
df_write.info()

<H1> Lastly, Encapsulate Strings with quotes, Fill in NaN values with 0

In [ ]:
df_write.to_csv('IEPPRO_Draft_forUpload.csv', sep='|', index=False, encoding='utf-8')

In [ ]:
df_write_noblanks.to_csv('IEPPRO_DataToValidate.csv', sep='|', index=False, encoding='utf-8')